In [1]:
from GAME.bin.intertask_mappings import *
from GAME.utils.config import config
from itertools import product
from sklearn.preprocessing import MinMaxScaler

In [6]:
def agg_res(eval_results:dict, state:str=None, action:int=None, strategy:str='average') -> dict:
    agg_results = {}
    state_agg = []
    action_agg = []
    for k, v in eval_results.items():
        key_split = k.split('--')
        key_state = key_split[1].split('-')[1]
        key_action = key_split[0]
        if state and state == key_state:
            state_agg.append(v)
        if action and str(action) == key_action:
            action_agg.append(v)
    if strategy == 'average':
        if state:
            agg_results[state] = np.mean(state_agg)
        if action:
            agg_results[action] = np.mean(action_agg)
    if strategy == 'best':
        if state:
            agg_results[state] = np.max(state_agg)
        if action:
            agg_results[action] = np.max(action_agg)

    return agg_results

In [2]:
# load config data
config_data = config()

src_state_var_names = config_data['3v2_state_names']
src_action_names = config_data['3v2_action_names']
src_action_values = config_data['3v2_action_values']
target_state_var_names = config_data['4v3_state_names']
target_action_names = config_data['4v3_action_names']
target_action_values = config_data['4v3_action_values']

# mapping = IntertaskMapping([0 for _ in range(19)], [0, 1, 2, 2], src_state_var_names, src_action_names, target_state_var_names, target_action_names)
# mapping = IntertaskMapping([11, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12], [0, 1, 2, 2], src_state_var_names, src_action_names, target_state_var_names, target_action_names)

# src_data_path = os.path.join(config_data['data_path'], 'keepaway', "keepaway_3v2_transitions.csv")
src_data_path = os.path.join(config_data['logs_path'], '12142022_3v2_logs_random_explore', "keepaway_3v2_transitions.csv")
src_data_df = pd.read_csv(src_data_path, index_col = False)
transformed_df_col_names = config_data['4v3_full_transition_df_col_names']

feature_scaler = MinMaxScaler()
feature_names = ['Current-{}'.format(feature) for feature in src_state_var_names]
feature_scaler.fit(src_data_df[feature_names])
src_data_df[feature_names] = feature_scaler.transform(src_data_df[feature_names])

target_scaler = MinMaxScaler()
target_names = ['Next-{}'.format(feature) for feature in src_state_var_names]
target_scaler.fit(src_data_df[target_names])
src_data_df[target_names] = target_scaler.transform(src_data_df[target_names])

# src_data_df['Current-action'] = src_data_df['Current-action'].replace(0, 4)
# src_data_df['Current-action'] = src_data_df['Current-action'].replace(1, 0)
# src_data_df['Current-action'] = src_data_df['Current-action'].replace(4, 1)

# transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)
# transformed_df_out_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\output\\11022022 Transformed Source Data 3DMC\\transformed_df.csv"
# transformed_df.to_csv(transformed_df_out_path, index = False)

network_folder_path = os.path.join(config_data['pickle_path'], 'neural_nets', "keepaway")
eval_networks = EvaluationNetworks(network_folder_path)
# test_mlp = eval_networks.get_network(0, 'Next_x_position')

transformed_df_current_state_cols = config_data['4v3_current_state_transition_df_col_names']
transformed_df_next_state_cols = config_data['4v3_next_state_transition_df_col_names']

# eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [19]:
src_data_df.head()

,Cycle,ID_kp_w_ball,"Current-dist(K1,C)","Current-dist(K1,K2)","Current-dist(K1,K3)","Current-dist(K1,T1)","Current-dist(K1,T2)","Current-dist(K2,C)","Current-dist(K3,C)","Current-dist(T1,C)",...,"Next-dist(K1,T1)","Next-dist(K1,T2)","Next-dist(K2,C)","Next-dist(K3,C)","Next-dist(T1,C)","Next-dist(T2,C)","Next-Min(dist(K2,T1),dist(K2,T2))","Next-Min(dist(K3,T1),dist(K3,T2))","Next-Min(ang(K2,K1,T1),ang(K2,K1,T2))","Next-Min(ang(K3,K1,T1),ang(K3,K1,T2))"
0,12,1,0.432758,0.409361,0.430245,0.613692,0.670916,0.540005,0.704486,0.221195,...,0.278456,0.297748,0.457939,0.557291,0.180614,0.239681,0.151050,0.394805,0.277039,0.281338
1,13,1,0.429493,0.405694,0.430267,0.588360,0.642853,0.539967,0.704424,0.200811,...,0.255767,0.292698,0.457917,0.557272,0.146741,0.231204,0.143505,0.376677,0.280274,0.280290
2,14,1,0.428332,0.404261,0.430354,0.540418,0.631950,0.539941,0.704400,0.163150,...,0.229050,0.272146,0.457912,0.557260,0.110444,0.196403,0.133067,0.362074,0.281800,0.279236
3,15,1,0.427831,0.403706,0.430350,0.483968,0.587578,0.539935,0.704384,0.122794,...,0.218573,0.264344,0.427517,0.557260,0.097137,0.183449,0.125786,0.356749,0.282109,0.278485
4,16,1,0.427652,0.368297,0.430368,0.461831,0.570732,0.504096,0.704384,0.107999,...,0.223423,0.270514,0.390320,0.557260,0.092043,0.178427,0.120667,0.354342,0.274175,0.277545


In [3]:
state_mapping = [0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12]
# state_mapping = [0 for _ in range(19)]
action_mapping = [0, 1, 2, 2]
mapping = IntertaskMapping(state_mapping, action_mapping, src_state_var_names, src_action_names, target_state_var_names, target_action_names)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [4]:
print(parse_mapping_eval_scores(eval_results))

[0.9924721830482245]


In [7]:
for state in target_state_var_names:
    print(agg_res(eval_results, state, None, 'average'))

{'dist(K1,C)': 0.9947346068995139}
{'dist(K1,K2)': 0.9827432426498146}
{'dist(K1,K3)': 0.9917120682636412}
{'dist(K1,K4)': 0.9946253868274331}
{'dist(K1,T1)': 0.9962248021479843}
{'dist(K1,T2)': 0.9960782172222765}
{'dist(K1,T3)': 0.9958264574897349}
{'dist(K2,C)': 0.9931551139784565}
{'dist(K3,C)': 0.9951611823472221}
{'dist(K4,C)': 0.9904547988404464}
{'dist(T1,C)': 0.9943492608399588}
{'dist(T2,C)': 0.9932303961455623}
{'dist(T3,C)': 0.9931134019966208}
{'Min(dist(K2,T1),dist(K2,T2),dist(K2,T3))': 0.9921385788532131}
{'Min(dist(K3,T1),dist(K3,T2),dist(K3,T3))': 0.9899792175293123}
{'Min(dist(K4,T1),dist(K4,T2),dist(K4,T3))': 0.9751072004194791}
{'Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3))': 0.995566121806506}
{'Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3))': 0.9958710404924019}
{'Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3))': 0.996900383166696}


In [16]:
# GAME
state_mapping = [0, 11, 5, 0, 4, 3, 4, 9, 6, 6, 7, 9, 9, 8, 2, 4, 11, 10, 9]
# state_mapping = [0 for _ in range(19)]
action_mapping = [0, 0, 0, 2]
mapping = IntertaskMapping(state_mapping, action_mapping, src_state_var_names, src_action_names, target_state_var_names, target_action_names)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [17]:
print(parse_mapping_eval_scores(eval_results))

[0.9956558299018758]


In [10]:
for state in target_state_var_names:
    print(agg_res(eval_results, state, None, 'average'))

{'dist(K1,C)': 0.9971359718007959}
{'dist(K1,K2)': 0.9974760746694861}
{'dist(K1,K3)': 0.9964188333028199}
{'dist(K1,K4)': 0.9976108898780987}
{'dist(K1,T1)': 0.9981445915858206}
{'dist(K1,T2)': 0.9978469746729501}
{'dist(K1,T3)': 0.9975934438060942}
{'dist(K2,C)': 0.9976508462699196}
{'dist(K3,C)': 0.9963842288912367}
{'dist(K4,C)': 0.9930856446026871}
{'dist(T1,C)': 0.9980757516808986}
{'dist(T2,C)': 0.9980616695365634}
{'dist(T3,C)': 0.998275185798032}
{'Min(dist(K2,T1),dist(K2,T2),dist(K2,T3))': 0.9962481236731847}
{'Min(dist(K3,T1),dist(K3,T2),dist(K3,T3))': 0.9949288670516651}
{'Min(dist(K4,T1),dist(K4,T2),dist(K4,T3))': 0.9969513634504943}
{'Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3))': 0.9961090211517939}
{'Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3))': 0.9978656546564274}
{'Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3))': 0.998204062891829}


In [11]:
# GAME
state_mapping = [0, 12, 9, 1, 4, 4, 4, 1, 11, 9, 9, 9, 9, 4, 9, 9, 9, 11, 9]
# state_mapping = [0 for _ in range(19)]
action_mapping = [0, 0, 0, 0]
mapping = IntertaskMapping(state_mapping, action_mapping, src_state_var_names, src_action_names, target_state_var_names, target_action_names)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [12]:
print(parse_mapping_eval_scores(eval_results))

[0.9971511200395693]


In [13]:
for state in target_state_var_names:
    print(agg_res(eval_results, state, None, 'average'))

{'dist(K1,C)': 0.9971826045145181}
{'dist(K1,K2)': 0.9974442869777455}
{'dist(K1,K3)': 0.9977925177529048}
{'dist(K1,K4)': 0.9983232479220194}
{'dist(K1,T1)': 0.9975912307068051}
{'dist(K1,T2)': 0.9976583205774555}
{'dist(K1,T3)': 0.9967093667067376}
{'dist(K2,C)': 0.9980169062794082}
{'dist(K3,C)': 0.9927059271383515}
{'dist(K4,C)': 0.9981392265560491}
{'dist(T1,C)': 0.9982688300186352}
{'dist(T2,C)': 0.9979816438780178}
{'dist(T3,C)': 0.9983983932965935}
{'Min(dist(K2,T1),dist(K2,T2),dist(K2,T3))': 0.994562353847447}
{'Min(dist(K3,T1),dist(K3,T2),dist(K3,T3))': 0.997890110514019}
{'Min(dist(K4,T1),dist(K4,T2),dist(K4,T3))': 0.993753667582993}
{'Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3))': 0.9978967682220206}
{'Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3))': 0.9970785583052031}
{'Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3))': 0.9984773199549046}
